# Описание проекта

* Компания: "Хоть потоп"
* Сфера деятельности: страхование

* Цель: построить модель линейной регрессии, прогнозирующую количество страховых выплат.

* Задачи:
    - Предложить алгоритм преобразования данных с целью защиты персональных свеедений, который не повлияет на модель линейной регрессии;
    - Построить модели на непреобразованных и преобразованных данных и показать, что R^2 не отличается;
    - Дать теоретическое обоснование работы данного алгоритма.

* Данные о клиентах страховой компании:
    - Пол
    - Возраст
    - Заработная плата застрахованного
    - Количество членов семьи

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from numpy.random import RandomState
from sklearn.metrics import r2_score

## 1. Загрузка и подготовка данных к анализу

### 1.1 Загрузка  данных

In [2]:
# Загрузка данных
try:
    df = pd.read_csv('insurance.csv')
except:
    df = pd.read_csv('/datasets/insurance.csv')
    
print('Вывод первых 5 строк из таблицы с данными.')
display(df.head())
print()
print('Общая информация')
df.info()

Вывод первых 5 строк из таблицы с данными.


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0



Общая информация
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
# Для удобства переименуем столбцы
df.columns = ['sex', 'age', 'wage', 'n_family', 'n_insurance_payments']

### 1.2 Подготовка данных к анализу

In [4]:
# Выделим признаки и целевой показатель
features = df.drop('n_insurance_payments', axis = 1)
target = df['n_insurance_payments']

### Краткие выводы по разделу 1
* Данные выгружены корректно.
* В данных отсуствуют пропуски.
* Типы данных соответствуют смыслу этих характеристик.
* Для удобства название характеристик (столбцов) были зменены, чтобы не использовать кириллицу.
* **Данные не требуют специальной предобработки**.
* Данные были разбиты на характеристики и целевой признак

In [5]:
# Описательная статистика
df.describe()

,sex,age,wage,n_family,n_insurance_payments
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
# Матрица корреляции
df.corr()

,sex,age,wage,n_family,n_insurance_payments
sex,1.000000,0.002074,0.014910,-0.008991,0.010140
age,0.002074,1.000000,-0.019093,-0.006692,0.651030
wage,0.014910,-0.019093,1.000000,-0.030296,-0.014963
n_family,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
n_insurance_payments,0.010140,0.651030,-0.014963,-0.036290,1.000000


## 2. Алгоритм защиты данных

### 2.1 Умножение признаков на обратимую (невырожденную) матрицу

Из теории линейной алгебры следует, что необходимым и достаточным условием обратимости матрицы является **НЕРАВЕНСТВО определителя матрицы 0**. Поэтому создадим случайную матрицу, у которой определитель не будет равным 0.

In [7]:
# Создадим случайную обратимую матрице для линейного преобразования признаков

# Сначала создаем нулевую квадратную матрицу размером 4х4
# Определитель этой матрицы равняется 0, т. е. для нее не существует обратной.
encoding_matrix = np.zeros((features.shape[1], features.shape[1]))

# Счетчик итераций подбора случайной матрицы.
i=0

# Хоть вероятность того, что случайная матрица окажется вырожденной
# крайне мала, все же стоит избежать этой случайности и запустить цикл
while np.linalg.det(encoding_matrix) == 0:
    random_state = RandomState(12345+i) # фиксируем случайное состояние (пока рано заниматься динамическим шифрованием)
    encoding_matrix = random_state.random(size=(features.shape[1], features.shape[1]))
    i += 1
 
decoding_matrix = np.linalg.inv(encoding_matrix)
print('Общее число итераций подбора случайной невырожденной матрицы:', i)
print()
print('Матрица линейного преобразования признаков:')
print(encoding_matrix)
print()
print(f'Определитель матрицы равен: {np.linalg.det(encoding_matrix):.2}')
print()
print('Обратная матрица равна (матрица дешифровки данных):')
print(decoding_matrix)

Общее число итераций подбора случайной невырожденной матрицы: 1

Матрица линейного преобразования признаков:
[[0.92961609 0.31637555 0.18391881 0.20456028]
 [0.56772503 0.5955447  0.96451452 0.6531771 ]
 [0.74890664 0.65356987 0.74771481 0.96130674]
 [0.0083883  0.10644438 0.29870371 0.65641118]]

Определитель матрицы равен: -0.055

Обратная матрица равна (матрица дешифровки данных):
[[ 2.1336149   0.5051926  -1.71113385  1.33832486]
 [-3.76534462 -2.61352053  6.72316006 -6.07193701]
 [ 0.97426298  2.9928115  -3.49875049  1.84220045]
 [ 0.13998298 -0.94454066  0.52375812  1.652661  ]]


### 2.2 Доказательство невлияния на метрику

В данном разделе мы будем пользоваться свойствами операторов транспонирования и свойствами обратных матриц.


Пусть есть модель линейной регрессии ($w_0$ включено в матрицу $w$):
$$\hat{y} = w*x$$
Веса в этой модели будут равны:
$$w^* = (x^T*x)^{-1}*x^T*y$$


Теперь рассмотрим другую модель с зашифрованной матрицей признаков путем линейного преобразования:
$$\tilde{y} = \tilde{w}*\tilde{x}$$

где $\tilde{x} = x*C$ (C -  квадратная невырожденная матрица преобразования).
Тогда веса новой модели будут рассчитаны следующим образом:

$$\tilde{w^*} = (\tilde{x}^T*\tilde{x})^{-1}*\tilde{x}^T*y$$

Коэффициент детерминации - это доля десперсии модели от дисперсии истинных значений. Доля десперсии модели  - это дисперсия отклонений предсказаний модели от истинных значений. Если мы покажем, предсказания обеих моделей совпадают, то и метрики этих моделей будут совпадать.

$$\hat{\tilde{y}} = \tilde{w}*\hat{\tilde{x}} = (\tilde{x}^T*\tilde{x})^{-1}*\tilde{x}^T*y*\tilde{x} = ((x*C)^T*x*C)^{-1}*(x*C)^T*y*x*C $$

Рассмотрим отдельно:
$$((x*C)^T*x*C)^{-1} = [C^T*(x^T*x)*C]^{-1} = C^{-1}*(x^T*x)*(C^T)^{-1}$$

Подставляем данное выражение и получаем:

$$((x*C)^T*x*C)^{-1}*(x*C)^T*y*x*C =$$ 

$$= C^{-1}*(x^T*x)*(C^T)^{-1}*C^T*x^T*y*x*C = C^{-1}*(x^T*x)*x^T*y*x*C = $$
$$=C^{-1}*w*x*C = C^{-1}*\hat{y}*C = \hat{y}$$

Таким образом, предсказания обеих моделей (с шифровкой и без совпадают, а значит и метрики будут совпадать)

### Краткие выводы по разделу 2
* В качестве алгоритма шифрования выбрано линейное преобразование признаков путем матричного умножения
* Случайным образом определена матрица линейного преобразования
* Было доказано, что линейное преобразование признаков не меняет значение коэффициента детерминации

## 3. Шифровка данных (исходных признаков)

In [8]:
# Шифруем исходные признаки с помощью матрицы преобразования сохраняем в отдельную переменную
features_encoded = features.values @ encoding_matrix
features_encoded

array([[37169.98395227, 32441.905747  , 37126.68225769, 47708.45534165],
       [28484.57596592, 24863.15659308, 28457.82912393, 36560.35852841],
       [15743.50341406, 13742.23808427, 15729.98191577, 20206.38359334],
       ...,
       [25399.30628958, 22168.14240469, 25367.41973212, 32602.67471717],
       [24502.69177899, 21385.472469  , 24472.57361256, 31451.27395954],
       [30422.44378908, 26552.0348282 , 30384.71028549, 39048.20341473]])

In [9]:
# ПРОВЕРКА: дешифровка признаков
round(pd.DataFrame(features_encoded @ decoding_matrix), 1)

,0,1,2,3
0,1.0,41.0,49600.0,1.0
1,-0.0,46.0,38000.0,1.0
2,0.0,29.0,21000.0,0.0
3,-0.0,21.0,41700.0,2.0
4,1.0,28.0,26100.0,-0.0
...,...,...,...,...
4995,-0.0,28.0,35700.0,2.0
4996,-0.0,34.0,52400.0,1.0
4997,-0.0,20.0,33900.0,2.0
4998,1.0,22.0,32700.0,3.0


### Краткие выводы по разделу 3
* Шифровка данных с помощью линейного преобразования корректна, но неточности возникают в силу принципов округления чисел в питоне

## 4. Построение моделей линейных регрессий

### 4.1 Построение линейной регрессии на исходных (незашифрованных) данных

In [10]:
# Построение модели
model = LinearRegression()
model = model.fit(features, target)

# По умолчанию оценка линейной регрессии это коэффициент детерминации
r2 = model.score(features, target)
print('Коэффициент детерминации модели на исходных данных равен:', r2)

Коэффициент детерминации модели на исходных данных равен: 0.4249455028666801


### 4.2 Построение линейной регрессии на зашифрованных данных

In [11]:
model_encoded = LinearRegression()
model_encoded = model.fit(features_encoded, target)
r2_encoded = model.score(features_encoded, target)
print('Коэффициент детерминации модели на зашифрованных данных равен:', r2_encoded)

Коэффициент детерминации модели на зашифрованных данных равен: 0.4249455028667376


In [17]:
# Функция сравнения чисел сточностью до заданного порядка
def compare_numbers(a, b, order=45):
    i=0
    while (round(a, i) == round(b, i)) and (i<order):
        i += 1
    
    if i>0:
        print(f'Числа равны с точностью {i-1} знаков после запятой.')
    else:
        print(f'Числа не равны.')
        

In [18]:
compare_numbers(np.e, np.pi)

Числа равны с точностью 0 знаков после запятой.


In [38]:
compare_numbers(r2, r2_encoded, 13)

Числа равны с точностью 14 знаков после запятой.


<div class="alert alert-success">
<b>✔️ Комментарий ревьювера:</b> Качество предсказания действительно не изменяется, результат достигнут!
    
Еще могу порекомендовать вот такой материал:
    
https://habr.com/ru/post/479398/

https://habr.com/ru/company/vk/blog/478514/ Применение зашифрованных данных для машинного обучения

https://www.amazon.science/blog/machine-learning-models-that-act-on-encrypted-data#:~:text=Privacy%2Dpreserving%20machine%20learning%20 Privacy-preserving machine learning (PPML)
 
</div>

### Краткие выводы по разделу 4
* Оценены линейные регрессии на исходных и зашифрованных данных.
* С точностью до 14 знаков после запятой коэффициенты детерминации этих моделей равны (неточность возникает в силу особенностьей округления в Python)

## Общие выводы
* Линейное преобразование признаков не сказывается на качестве моделей линейной регрессии
* Коэффициент детерминации в модели составил порядка 0,425. Это означает, что примерно 42,5% вариации числа страховых выплат можно объяснить представленными признаками (пол, возраст, зарплата, число челенов семьи)

<font color=red>
<b> ✔️ 👍Заключение ревьювера:</b> Константин, было приятно проверять твою работу, она выполнена на очень хорошем уровне! 
   
Могу отметить грамотно составленный код, хорошую практическую часть, доказательство без ошибок =)
    
Замечаний по проекту нет, я отметил только рекомендации!
    
Отправляю работу для ознакомления с ходом проверки, а также если есть вопросы — смело задавай их, я постараюсь тебе помочь 😊

 <font> Спасибо за дополнительные материалы. Со временем я их обязательно изучу.</font>    

Жду твоего ответа!
    

    
</font>

   

<div class="alert alert-success">
<b>✔️ Комментарий ревьювера (v2):</b> Да, сохрани на будущее, как будет потребность обратишься к ссылкам =)
    
Желаю успехов в дальнейшем обучении!
    
</div>